# Summary

实验结果：

在 CIFAR10 数据集上的实验结果，使用 GPU GTX 1080.

| 模型 | 正确率 | 运行时间 |
| -- | -- | -- |
| AlexNet | 70% | 45s |
| VggNet | 78% | 500s |

观点：

1. VggNet 成功的把网络做的更深了，模型的算法性能也有了不错的提升。
2. CIFAR10 数据集可能不足以体现 VggNet 的优势，需要换 ImageNet 试一下

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
import time

tf.__version__

'2.1.0'

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
cifar10 = tf.keras.datasets.cifar10
(x_train, y_train),(x_test, y_test) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

In [4]:
x_train.shape

(50000, 32, 32, 3)

In [5]:
def res_net_block(input_data, filters, conv_size):
    x = layers.Conv2D(filters, conv_size, activation='relu', padding='same')(input_data)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters, conv_size, activation=None, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([x, input_data])
    x = layers.Activation('relu')(x)
    return x

In [7]:
N, W, H, C = x_train.shape
N_LABELS = 10

inputs = tf.keras.Input(shape=(W, H, C))
x = layers.Conv2D(32, 3, activation='relu')(inputs)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(3)(x)

num_res_net_blocks = 10
for i in range(num_res_net_blocks):
  x = res_net_block(x, 64, 3)

x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

In [8]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
print('!! start time counting')
tic = time.time()

history = model.fit(x_train, y_train, epochs=10, 
                    validation_data=(x_test, y_test))
model.evaluate(x_test, y_test, verbose=2)
toc = time.time()
print('!! Total timecost: %.2fs' % (toc - tic))

!! start time counting
Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 20s 400us/sample - loss: 1.5689 - accuracy: 0.4235 - val_loss: 1.3029 - val_accuracy: 0.5315
Epoch 2/10
50000/50000 [==============================] - 17s 333us/sample - loss: 1.1415 - accuracy: 0.5947 - val_loss: 1.4754 - val_accuracy: 0.4920
Epoch 3/10
50000/50000 [==============================] - 17s 331us/sample - loss: 0.9368 - accuracy: 0.6773 - val_loss: 1.3205 - val_accuracy: 0.5733
Epoch 4/10
50000/50000 [==============================] - 17s 337us/sample - loss: 0.7941 - accuracy: 0.7310 - val_loss: 0.9169 - val_accuracy: 0.6843
Epoch 5/10
50000/50000 [==============================] - 17s 334us/sample - loss: 0.7058 - accuracy: 0.7615 - val_loss: 0.8991 - val_accuracy: 0.7011
Epoch 6/10
50000/50000 [==============================] - 17s 335us/sample - loss: 0.6323 - accuracy: 0.7879 - val_loss: 0.8129 - val_accuracy: 0.7359
Epoch 7/10
50000/5000

In [ ]:
def non_res_block(input_data, filters, conv_size):
  x = layers.Conv2D(filters, conv_size, activation='relu', padding='same')(input_data)
  x = layers.BatchNormalization()(x)
  x = layers.Conv2D(filters, conv_size, activation='relu', padding='same')(x)
  x = layers.BatchNormalization()(x)
  return x

In [13]:
# 3x3 convolution
def conv3x3(channels, stride=1, kernel=(3, 3)):
    return layers.Conv2D(channels, kernel, strides=stride, padding='same',
                               use_bias=False,
                            kernel_initializer=tf.random_normal_initializer())

class ResnetBlock(Model):

    def __init__(self, channels, strides=1, residual_path=False):
        super(ResnetBlock, self).__init__()

        self.channels = channels
        self.strides = strides
        self.residual_path = residual_path

        self.conv1 = conv3x3(channels, strides)
        self.bn1 = layers.BatchNormalization()
        self.conv2 = conv3x3(channels)
        self.bn2 = layers.BatchNormalization()

        if residual_path:
            self.down_conv = conv3x3(channels, strides, kernel=(1, 1))
            self.down_bn = layers.BatchNormalization()

    def call(self, inputs, training=None):
        residual = inputs

        x = self.bn1(inputs, training=training)
        x = tf.nn.relu(x)
        x = self.conv1(x)
        x = self.bn2(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv2(x)

        # this module can be added into self.
        # however, module in for can not be added.
        if self.residual_path:
            residual = self.down_bn(inputs, training=training)
            residual = tf.nn.relu(residual)
            residual = self.down_conv(residual)

        x = x + residual
        return x

In [ ]:
class ResNet(Model):

    def __init__(self, block_list, initial_filters=16, **kwargs):
        super(ResNet, self).__init__(**kwargs)

        self.num_blocks = len(block_list)
        self.block_list = block_list

        self.in_channels = initial_filters
        self.out_channels = initial_filters
        self.conv_initial = conv3x3(self.out_channels)

        self.blocks = models.Sequential(name='dynamic-blocks')

        # build all the blocks
        for block_id in range(len(block_list)):
            for layer_id in range(block_list[block_id]):

                if block_id != 0 and layer_id == 0:
                    block = ResnetBlock(self.out_channels, strides=2, residual_path=True)
                else:
                    if self.in_channels != self.out_channels:
                        residual_path = True
                    else:
                        residual_path = False
                    block = ResnetBlock(self.out_channels, residual_path=residual_path)

                self.in_channels = self.out_channels

                self.blocks.add(block)

            self.out_channels *= 2

        self.final_bn = keras.layers.BatchNormalization()
        self.avg_pool = keras.layers.GlobalAveragePooling2D()
        self.fc = keras.layers.Dense(num_classes)

    def call(self, inputs, training=None):

        out = self.conv_initial(inputs)

        out = self.blocks(out, training=training)

        out = self.final_bn(out, training=training)
        out = tf.nn.relu(out)

        out = self.avg_pool(out)
        out = self.fc(out)


        return out

In [4]:
# using Batch Norm

cifar10 = tf.keras.datasets.cifar10
(x_train, y_train),(x_test, y_test) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

N, W, H, C = x_train.shape
N_LABELS = 10
initial_filters=16

weight_decay = 0.000

model = models.Sequential()

model.add(layers.Conv2D(16, (3, 3), strides=1, padding='same',
                 input_shape=(W, H, C), kernel_initializer=tf.random_normal_initializer()))


# //////// starting
# 1st max pooling. 2 conv layer
model.add(layers.Conv2D(64, (3, 3), padding='same',
                 input_shape=(W, H, C), kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))

model.add(layers.Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())

model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# 2nd max pooling. 2 conv layer
model.add(layers.Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())

model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# 3rd max pooling. 3 conv layer
model.add(layers.Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())

model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# 4th max pooling. 3 conv layer
model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())

model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# 5th max pooling. 3 conv layer
model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())

model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.5))

# 2 Fully Connected Layers
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.BatchNormalization())

model.add(layers.Dropout(0.5))
model.add(layers.Dense(N_LABELS, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
print('!! start time counting')
tic = time.time()

history = model.fit(x_train, y_train, epochs=10, 
                    validation_data=(x_test, y_test))
model.evaluate(x_test, y_test, verbose=2)
toc = time.time()
print('!! Total timecost: %.2fs' % (toc - tic))

!! start time counting
Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 52s 1ms/sample - loss: 2.0852 - accuracy: 0.2730 - val_loss: 1.5258 - val_accuracy: 0.4365
Epoch 2/10
50000/50000 [==============================] - 49s 985us/sample - loss: 1.3984 - accuracy: 0.4961 - val_loss: 1.3332 - val_accuracy: 0.5622
Epoch 3/10
50000/50000 [==============================] - 49s 985us/sample - loss: 1.1125 - accuracy: 0.6151 - val_loss: 1.2190 - val_accuracy: 0.6067
Epoch 4/10
50000/50000 [==============================] - 49s 986us/sample - loss: 0.9724 - accuracy: 0.6727 - val_loss: 0.8789 - val_accuracy: 0.7029
Epoch 5/10
50000/50000 [==============================] - 49s 988us/sample - loss: 0.8501 - accuracy: 0.7203 - val_loss: 0.8442 - val_accuracy: 0.7317
Epoch 6/10
50000/50000 [==============================] - 49s 988us/sample - loss: 0.7547 - accuracy: 0.7558 - val_loss: 0.6965 - val_accuracy: 0.7688
Epoch 7/10
50000/50000 

In [ ]:
# train vs evaluate 的 accuracy 曲线
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(test_acc)